## `1. Introduction`

#### Text -> Word (Tokens)

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
import re

preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4649


In [4]:
print(preprocessed[70:100])

[')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter', '--']


#### Word -> Token IDs 

In [5]:
# Create vocabulary
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

1159


In [6]:
vocab = {token: index for index, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i > 5:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)


## `2. Implementing a simple tokenizer`

In [15]:
import re

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {index: text for text, index in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[item] for item in preprocessed]

        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[id] for id in ids])
        return text

In [16]:
tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know," Mrs. Gisburn said with."""
ids = tokenizer.encode(text)
print(ids)

[58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 7]


In [17]:
print(tokenizer.decode(ids))

It ' s the last he painted , you know , " Mrs . Gisburn said with .


Giờ ta sẽ thử với một đoạn text mà không có trong tập training:

In [18]:
text = "Hello, do you like tea?"
tokenizer.encode(text)

KeyError: 'Hello'

- Vấn đề là từ "Hello" không nằm trong *The Verdict* nên sẽ không có trong *vocabulary*. Điều này cho thấy cần phải chuẩn bị 1 tập training lớn và đa dạng hơn để mở rộng cho *vocabulary*.

- Ở phần sau, ta sẽ tiếp tục thử nghiệm Tokenizer trên văn bản chứa những từ mà *vocabulary* chưa có, đồng thời thảo luận về các `token đặc biệt` bổ sung có thể được sử dụng để cung cấp thêm ngữ cảnh cho LLM trong quá trình training. 

## `3. Adding special context tokens`

In [22]:
all_tokens = sorted(list(set(preprocessed)))        # List of all unique tokens
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: index for index, token in enumerate(all_tokens)}

print(len(vocab.items()))

1161


In [23]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)


`A simple text tokenizer that handles unknown words`

In [25]:
import re

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {index: text for text, index in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[item] for item in preprocessed]

        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[id] for id in ids])

        # Remove space before punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)

        return text

In [26]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)


Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [27]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))


[1160, 5, 362, 1155, 642, 1000, 10, 1159, 57, 1013, 981, 1009, 738, 1013, 1160, 7]


Có thể thấy ở trên có ID 1160 cho token `<|unk|>` và 1159 cho token `<|endoftext|>`

Giờ hãy *decode* để xem lại text gốc

In [28]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


In [9]:
# import requests, csv, io
# import re

# url = "https://docs.google.com/spreadsheets/d/1BZqhx0lXy107MasuLkLdjXaR319WWKXz9GLGjWg_60Y/export?format=xlsx"
# resp = requests.get(url, timeout=60)
# excel_content = resp.content

# with open("complete_workbook.xlsx", "wb") as f:
#     f.write(excel_content) 

In [ ]:
# import pandas as pd

# df = pd.read_excel("complete_workbook.xlsx", sheet_name="229 - 279", skiprows=2, header=[0, 1])

# # Làm sạch level 2: đổi 'Unnamed: ...' -> "" (trống)
# lvl0 = df.columns.get_level_values(0).astype(str)
# lvl1 = df.columns.get_level_values(1).astype(str).map(
#     lambda s: "" if s.startswith("Unnamed:") else s
# )

# # Gộp 2 level thành 1 tên cột phẳng
# new_cols = []
# for a, b in zip(lvl0, lvl1):
#     a = re.sub(r"\s+", " ", a).strip()          # bỏ xuống dòng/thừa khoảng trắng
#     b = re.sub(r"\s+", " ", b).strip()
#     new_cols.append(a if b == "" else f"{a}_{b}")

# df.columns = new_cols

# # (Tuỳ chọn) xử lý trùng tên cột nếu có
# df = df.loc[:, ~df.columns.duplicated()]

# # ---- Lọc GIÁO VIÊN = 'Thảo Ly' (không phân biệt hoa thường, bỏ khoảng trắng thừa) ----
# mask = df['GIÁO VIÊN'].astype(str).str.strip().str.casefold() == 'Thảo Ly'.casefold()
# filtered = df[mask]

# print(filtered.head())

# # Nếu bạn muốn lấy dạng "tên_cột - value" cho từng dòng:
# result = [
#     [f"{col} - {row[col]}" for col in filtered.columns]
#     for _, row in filtered.iterrows()
# ]
# # print(df.head())

# # df[df['GIÁO VIÊN'] == 'Thảo Ly']
# # print(df.columns.tolist())

# # filtered = df[df['GIÁO VIÊN'] == 'THẢO LY']

# # result = []
# # for _, row in filtered.iterrows():
# #     for col, val in row.items():
# #         result.append(f"{col} - {val}")

# # print(result)

    STT GIÁO VIÊN Nơi ở (Quận cũ) 2- MON - 22/09_Sáng 2- MON - 22/09_Chiều  \
20   21   Thảo Ly         Thủ Đức                 NaN       TiH Từ Đức (2)   

    3- TUE - 23/09_Sáng                       3- TUE - 23/09_Chiều  \
20  THCS Bình Chiểu (3)  Chạy trường  An Bình (1) -  Bình Quới (1)   

   4- WED - 24/09_Sáng 4- WED - 24/09_Chiều 5- THURS - 25/09_Sáng  \
20                 NaN       TiH Từ Đức (3)                   NaN   

   5- THURS - 25/09_Chiều 6- FRI - 26/09_Sáng 6- FRI - 26/09_Chiều  \
20         TiH Từ Đức (3)                 NaN       TiH Từ Đức (3)   

   7- SAT - 27/09_Sáng  Tổng số tiết trong tuần_Tiểu học  \
20           LV ONLINE                               NaN   

    Tổng số tiết trong tuần_THCS  Tổng số tiết trong tuần_THPT  
20                           NaN                           NaN  
